#  Classification : Nearest Neighbors and Naive Bayes 

## Classification using Nearest Neighbors

(a) Perform k-Nearest neighbours on the given dataset($X_{knn}$ and $y_{knn}$: where $X_{knn}$ stores feature vectors representing the movies and  $y_{knn}$ stores the 0-1 labelling for each movie) for binary classification of movies, for classifiying whether a given movie is a comedy(label 1) or not a comedy(label 0) . Split the dataset into train(80%), validation(10%) and test sets(10%).Run k-Nearest neighbours for different k values (1,3,7,15,31,63). Select the k, using validation set, which returns the best accuracy score. 

(i)  Report all the validation accuracies for all the values of k. 
<br>(ii) Report accuracy score by performing k-NN on the test dataset using the best chosen k value. 

In [8]:
## write your code here.
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

X = np.loadtxt('X_knn.csv')
y = np.loadtxt('y_knn.csv')

X_train , X_test , y_train , y_test = train_test_split(X,y,test_size = 0.1, random_state = 42)
X_train , X_validation , y_train , y_validation = train_test_split(X_train,y_train,test_size = 1/9,random_state = 42)

clf_list = []
score_list = []
for k in [1,3,7,15,31,63] :
    clf = KNeighborsClassifier(k)
    clf.fit(X_train,y_train)
    score_list.append(clf.score(X_validation,y_validation))
    clf_list.append(clf)

for k,s in zip([1,3,7,15,31,63],score_list) :   
    print('Value of K :',k,'corresponding validation Accuracy :',s)    

Value of K : 1 corresponding validation Accuracy : 0.84
Value of K : 3 corresponding validation Accuracy : 0.853
Value of K : 7 corresponding validation Accuracy : 0.846
Value of K : 15 corresponding validation Accuracy : 0.848
Value of K : 31 corresponding validation Accuracy : 0.847
Value of K : 63 corresponding validation Accuracy : 0.85


In [11]:
print('We can see that from the validation accuracy, the value of 3 for k has the highest accuracy.')
print('For k value of 3, the accuracy on test is :',clf_list[1].score(X_test,y_test))

We can see that from the validation accuracy, the value of 3 for k has the highest accuracy.
For k value of 3, the accuracy on test is : 0.846


Write the results here

(b) State why using an even value of k in k-NN should not be chosen

We do not choose even values of k for k nearest neighbours because if even values were chosen, there could arise a posibility wherein during testing a data point for classification, in the closest k train data points, half of them could be belonging to one class and half could belong to the other class. In this situation, we cannot decide which class the test point belongs to. Therefore, in order to avoid such cases, we do not use even values of k. 

## Learning Naive Bayes' classifier  

### From Continuous Distribution of data

Here, the distribution of the data( $X$ represents the datapoints and $Y$ represents the 0-1 binary-class label; where 0 being the negative class and 1 being the positive class) is already known.
<br>Consider the following one-dimensional(1-D) Gaussian distributions where means and variances are unknown. You need to estimate means($\mu_-$: for negative class and  $\mu_+$: for positive class) and variances ($\sigma^{2}_{-}$: for negative class and $\sigma^{2}_+$: for positive class) from the given data : 
<br> (1) Assume $X|Y_{Y=0} \sim \mathcal{N}(\mu_- , \sigma^{2}_-)$ 
<br>(2) Assume $X|Y_{Y=1} \sim \mathcal{N}(\mu_+ , \sigma^{2}_+)$


*Generating artificial datasets in the next cell *

In [42]:
## This cell is for generating datasets. Students should not change anything in this cell. 
## You can compare your mean and variance estimates by the actual ones used to generate these datasets

import numpy as np
X_pos = np.random.randn(1000,1)+np.array([[2.]])
X_neg = np.random.randn(1000,1)+np.array([[4.]])
X_train_pos = X_pos[:900]
X_train_neg = X_neg[:900]
X_test_pos = X_pos[900:]
X_test_neg = X_neg[900:]
X_train = np.concatenate((X_train_pos, X_train_neg), axis=0)
X_test = np.concatenate((X_test_pos, X_test_neg), axis=0)
Y_train = np.concatenate(( np.ones(900),np.zeros(900) ))
Y_test = np.concatenate(( np.ones(100), np.zeros(100) ))

## X_train, X_test, Y_train, Y_test are your datasets to work with ####



<br>**Instructions to follow for learning a Baeysian classifier:** *(Code the formulae for estimating the different parameters yourself)*
<br> a)Utilize the training dataset to estimate the means($\hat{\mu_+}$,$\hat{\mu_-}$) and variances($\hat{\sigma^{2}_+}$, $\hat{\sigma^{2}_-}$) for both positive and negative classes  
b)Estimate the prior probability: $P(Y=1)$  ⟶ which could be referred to as: $\hat{a}$ 
<br>c)Estimate the classifier funtion/posterior probability:  $P(Y=1|X = x)$  ⟶ which could be referred to as $\hat{\eta(x)}$
<br>d)Find out the threshold value($x^*$) for classification by equating the estimated classifier function($\hat{\eta(x)}$)  with threshold probability of 0.5
<br>e)Classify the test dataset into the two classes using this threshold value($x^*$) and find out the **accuracy** of the prediction 

Return back:  $\hat{\mu_+}$, $\hat{\mu_-}$, $\hat{\sigma^{2}_+}$, $\hat{\sigma^{2}_-}$, $\hat{a}$, $x^*$ and accuracy from the code written 

*Hint: $X|Y_{Y=0} \sim \mathcal{N}(\mu_- , \sigma^{2}_-)$ implies $P_{X|Y=0} = \mathcal{N}(\mu_- , \sigma^{2}_-) $*


In [43]:
## write your code here.  

########## P(Y=1|X=x) = P(X=x|Y=1)/P(X=x)
########## Here, we have took that P(X=x|Y=1) is a gaussian
def eta_hat(x , mean_pos , var_pos , mean_neg , var_neg) :
    numerator = a_hat * ( np.exp( -(x-mean_pos)**2/var_pos )/np.sqrt(2*np.pi*var_pos) )
    denominator = a_hat * ( np.exp( -(x-mean_pos)**2/var_pos )/np.sqrt(2*np.pi*var_pos) ) + (1 - a_hat) * ( np.exp( -(x-mean_neg)**2/var_neg )/np.sqrt(2*np.pi*var_neg) )
    p = numerator / denominator
    return p
########## Calculating the mean and variance using the fact that the observations for Y=1 and Y=0 are independent
########## Hence, we can use law of large numbers and calculate the mean and corresponding variance for the train
########## sample.
mean_pos = np.sum(X_train[np.where(Y_train == 1)])/len(np.where(Y_train == 1)[0])
var_pos =  np.sum( (X_train[np.where(Y_train == 1)] - mean_pos)**2 )/(len(np.where(Y_train == 1)[0]))

mean_neg = np.sum(X_train[np.where(Y_train == 0)])/len(np.where(Y_train == 0)[0])
var_neg =  np.sum( (X_train[np.where(Y_train == 0)] - mean_neg)**2 )/(len(np.where(Y_train == 0)[0]))

########## P(Y=1) = total number of samples belonging to 1 / total number of train samples
a_hat = len(np.where(Y_train == 1)[0])/Y_train.shape[0]

########## To calculate x_star, we take a grid search on the x axis and calculate the eta_hat and take the x
########## which gives the closest value to 0.5
x = np.linspace( min(mean_pos,mean_neg) - 3*max(var_pos,var_neg),max(mean_pos,mean_neg) + 3*max(var_pos,var_neg),6000 )
eta_list = []
[eta_list.append(eta_hat(x[i],mean_pos,var_pos,mean_neg,var_neg)) for i in range(x.shape[0])]
x_star = x[np.argmin(np.abs(np.array(eta_list) - 0.5))]

Y_pred = []
for i in range(X_test.shape[0]) :
    if X_test[i] <= x_star :
        Y_pred.append(1)
    else :
        Y_pred.append(0)
Y_pred = np.array(Y_pred)        
acc = np.sum(Y_pred == Y_test)/Y_test.shape[0]

print('mu_positive :',mean_pos)
print('sigma_positive :',var_pos)
print('--------------------------------------')
print('mu_minus :',mean_neg)
print('sigma_negative :',var_neg)
print('--------------------------------------')
print('P(Y=1) :',a_hat)
print('x* :',x_star)
print('Accuracy on Test Data :',acc)

mu_positive : 2.0261587372544527
sigma_positive : 1.0299510787267079
--------------------------------------
mu_minus : 4.017737723461854
sigma_negative : 0.9817377058426409
--------------------------------------
P(Y=1) : 0.5
x* : 3.028077716033025
Accuracy on Test Data : 0.83


Write results here

As we can see, the observed mean and variance are close to the actual mean and variance. 

### From Discrete distribution of data

Unlike the first exercise for learning the Naive Bayes' classifier where we dealt with continuous distribution of data, here you need to work with discrete data, which means finding Probability Mass Distribution(PMF). 

Age  | Income | Status  | Buy
-----|--------|-------- |----
<=20 |  low   | students| yes
<=20 |  high  | students| yes
<=20 | medium | students| no
<=20 | medium | married | no
<=20 |  high  | married | yes
21-30|  low   | married | yes
21-30|  low   | married | no 
21-30| medium | students| no
21-30|  high  | students| yes
 >30 |  high  | married | no
 >30 |  high  | married | yes
 >30 | medium | married | yes
 >30 | medium | married | no
 >30 | medium | students| no
 
Consider the train dataset above. Take any random datapoint ($X_{i}$) where $X_{i} = (X_{i,1} = Age,X_{i,2} = Income,X_{i,3} = Status)$ and its corresponding label 

($Y_{i} = Buy$). A "yes" in Buy corresponds to label-1 and a "no" in Buy corresponds to label-0.

<br>**Instructions to follow for learning a Baeysian classifier:** *(Code the formulae for estimating the different parameters yourself)*
<br> a)Estimate the prior probability: $P(Y=1)$  ⟶ which could be referred to as: $\hat{a}$   
b)Estimate the likelihood for each feature:  $P(X_{i,j} = x |Y = y_{i})$, where $ i$=datapoint counter, $j \in \{1,2,3\}$ and $y_{i} \in \{0,1\}$ 
<br>c)Estimate the total likelihood: $P(X_{i} = x |Y = y_{i})$  
d)Calculate the posterior probability: $P(Y = 1|X_{i} = x_{test} )$ = $p_{test}$ where $x_{test} = (Age = 21-30, Income= medium, Status = married)$


Return back: $\hat{a}$, total likelihood and $p_{test}$ 


In [44]:
import pandas as pd

dict_data = {'Age' : ['<=20','<=20','<=20','<=20','<=20','21-30','21-30','21-30','21-30','>30','>30','>30','>30','>30'],
            'Income': ['low','high','medium','medium','high','low','low','medium','high','high','high','medium','medium','medium'],
            'Status': ['students','students','students','married','married','married','married','students','students','married','married','married','married','students'],
            'Buy' :['yes','yes','no','no','yes','yes','no','no','yes','no','yes','yes','no','no']}
dataset = pd.DataFrame(dict_data)

In [45]:
## write your code here.
def prob(df , age=None,income=None,Status=None,Buy=None,conditional=True):
    p = 1
    if age is not None and conditional == True:
        p = p * df.loc[df['Age'].isin(age) & df['Buy'].isin(Buy)].shape[0]/df.loc[df['Buy'].isin(Buy)].shape[0]
    if income is not None and conditional == True :
        p = p * (df.loc[df['Income'].isin(income) & df['Buy'].isin(Buy)]).shape[0]/df.loc[df['Buy'].isin(Buy)].shape[0]
    if Status is not None and conditional == True :    
        p = p * (df.loc[df['Status'].isin(Status) & df['Buy'].isin(Buy)]).shape[0]/df.loc[df['Buy'].isin(Buy)].shape[0]
    if Buy is not None and conditional == False :
        p = p * (df.loc[df['Buy'].isin(Buy)]).shape[0]/df.shape[0]
    return p 

age_list = ['<=20','21-30','>30']
age_dict = {'<=20':0 , '21-30' : 1 , '>30':2}
income_list = ['low','medium','high']
income_dict = {'low':0,'medium':1,'high':2}
status_list = ['students','married']
status_dict = {'students':0,'married':1}
buy_list = ['yes','no']
buy_dict = {'yes':1,'no':0}

####### P(Y=1)
a_hat = prob(dataset,Buy =['yes'],conditional=False)

####### Calulating the individual feature probability for each feature vector
prob_age = np.zeros((len(age_list),len(buy_list)))
for i in age_list :
    prob_age[age_dict[i],buy_dict['yes']] = prob(dataset,age=[i],Buy=['yes'])
    prob_age[age_dict[i],buy_dict['no']] = prob(dataset,age=[i],Buy=['no'])

prob_income = np.zeros((len(income_list),len(buy_list)))
for i in income_list :
    prob_income[income_dict[i],buy_dict['yes']] = prob(dataset,income=[i],Buy=['yes'])
    prob_income[income_dict[i],buy_dict['no']] = prob(dataset,income=[i],Buy=['no'])

prob_status = np.zeros((len(status_list),len(buy_list)))
for i in status_list :
    prob_status[status_dict[i],buy_dict['yes']] = prob(dataset,Status=[i],Buy=['yes'])
    prob_status[status_dict[i],buy_dict['no']] = prob(dataset,Status=[i],Buy=['no'])

####### Calculating the total likelihood making the assumption that the features are independent of each other
####### Hence, we can multiply them
prob_total_likelihood = np.zeros((len(age_list),len(income_list),len(status_list),len(buy_list)))
for i in age_list :
    for j in income_list : 
        for k in status_list :
            for m in buy_list :
                prob_total_likelihood[age_dict[i],income_dict[j],status_dict[k],buy_dict[m]] = prob(dataset,age=[i],Buy=[m])*prob(dataset,income=[j],Buy=[m])*prob(dataset,Status=[k],Buy=[m])


####### calculating P(Y=1|X=x). 
####### We calculate this using Bayes Theorem P(Y=1|X=x) = P(X=x|Y=1)P(Y=1)/(P(X=x|Y=1)P(Y=1) + P(X=x|Y=0)P(Y=0))
numerator = prob_total_likelihood[age_dict['21-30'],income_dict['medium'],status_dict['married'],buy_dict['yes']] * a_hat
denomenator = prob_total_likelihood[age_dict['21-30'],income_dict['medium'],status_dict['married'],buy_dict['no']]*(1-a_hat) + prob_total_likelihood[age_dict['21-30'],income_dict['medium'],status_dict['married'],buy_dict['yes']] * a_hat
p_test = numerator/denomenator

Write results here

In [46]:
print('P(Y=1) :',a_hat)

P(Y=1) : 0.5


In [47]:
print('p test :',p_test)

p test : 0.16666666666666663


In [48]:
print('Total Likelihood :')
print(prob_total_likelihood)
print('Axis : 0 - Age , 1 - Income , 2 - Status , 3 - Buy')

Total Likelihood :
[[[[0.01749271 0.05247813]
   [0.02332362 0.06997085]]

  [[0.08746356 0.02623907]
   [0.11661808 0.03498542]]

  [[0.01749271 0.10495627]
   [0.02332362 0.13994169]]]


 [[[0.01749271 0.03498542]
   [0.02332362 0.04664723]]

  [[0.08746356 0.01749271]
   [0.11661808 0.02332362]]

  [[0.01749271 0.06997085]
   [0.02332362 0.09329446]]]


 [[[0.02623907 0.03498542]
   [0.03498542 0.04664723]]

  [[0.13119534 0.01749271]
   [0.17492711 0.02332362]]

  [[0.02623907 0.06997085]
   [0.03498542 0.09329446]]]]
Axis : 0 - Age , 1 - Income , 2 - Status , 3 - Buy
